In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import MeCab
from gensim.models.doc2vec import Doc2Vec
import re
import glob

In [ ]:
def mygreeting(name):
    state = "hello [" + name + "]!!"
    print(state)

In [ ]:
def read_csv2df():
    df = pd.DataFrame()

    path = "csv/*.csv"
    for fpath in glob.glob(path):
        df_csv = pd.read_csv(fpath)
        df = pd.concat([df, df_csv]).reset_index(drop=True)
        
    return df

In [ ]:
def sanitize_minutes(df, col_name):
    for index, row in df.iterrows():
        val = row[col_name]
        val = re.sub('^.*■お問い合わせ内容（箇条書き）', '', val)
        val = re.sub('^.*■お問い合わせ内容', '', val)
        val = re.sub('^.*■Q1', '', val)
        val = re.sub('^.*■問：', '', val)
        val = re.sub('^.*■対応内容', '', val)
        val = re.sub('^.*■質問内容', '', val)
        val = re.sub('■回答内容', '', val)
        val = re.sub('■', '', val)
        df.at[index, col_name] = val
    return df

In [ ]:
def mecab_all_sep(text):
    words_list = mecab_sep(text, True)
    return words_list

In [ ]:
def mecab_sep(text, allflag):
    #m = MeCab.Tagger("-Owakati")
    m = MeCab.Tagger()
    node = m.parseToNode(text)
    
    words_list = []
    
    while node:
        word = ""
        if node.feature.split(",")[0] == "動詞":
            word = node.feature.split(",")[7]
            if word == "為る" or word == "つく":
                word = ""
        elif node.feature.split(",")[0] == "形容詞":
            word = node.feature.split(",")[7]
        elif node.feature.split(",")[0] == "名詞":
            word = node.surface
        elif (allflag):
            word = node.surface
        if word != "":
            words_list.append(word)
        node = node.next
        
    words_list = words_list[1:-1]
    # print(words_list)
    return words_list

In [ ]:
def myfunction(list1):
    list2 = list1[1:-1]
    return list2

In [ ]:
def calc_vecs(docs):
    vectorizer = TfidfVectorizer(analyzer=mecab_all_sep)
    vecs = vectorizer.fit_transform(docs)
    return vecs.toarray()